In [1]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np

def test_original_model():
    model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

    # 원본 모델과 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True
    )
    model.eval()

    PROMPT = '''You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''
    instructions = [
        "서울의 유명한 관광 코스를 만들어줄래?",
        "한국 전통 음식 중 외국인에게 추천할 만한 것은?",
        "기후 변화에 대응하기 위해 개인이 할 수 있는 일은?"
    ]

    results = []

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    for instruction in instructions:
        messages = [
            {"role": "system", "content": PROMPT},
            {"role": "user", "content": instruction}
        ]

        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        start_time = time.time()
        outputs = model.generate(
            input_ids,
            max_new_tokens=2048,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.1,
            no_repeat_ngram_size=3
        )
        end_time = time.time()

        generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

        inference_time = end_time - start_time
        token_count = len(tokenizer.encode(generated_text))
        tokens_per_second = token_count / inference_time

        result = {
            "instruction": instruction,
            "generated_text": generated_text,
            "inference_time": inference_time,
            "token_count": token_count,
            "tokens_per_second": tokens_per_second
        }
        results.append(result)

    return results

# 테스트 실행
results = test_original_model()

print(f"{'='*50}")
print("Original Model Evaluation Results")
print(f"{'='*50}")

avg_inference_time = np.mean([r["inference_time"] for r in results])
avg_tokens_per_second = np.mean([r["tokens_per_second"] for r in results])

print(f"Average Inference Time: {avg_inference_time:.2f} seconds")
print(f"Average Tokens/Second: {avg_tokens_per_second:.2f}")

print("\nFull Outputs:")
for idx, result in enumerate(results, 1):
    print(f"\n{'-'*40}")
    print(f"Sample {idx}:")
    print(f"Instruction: {result['instruction']}")
    print(f"Generated Text:\n{result['generated_text']}")
    print(f"Inference Time: {result['inference_time']:.2f} seconds")
    print(f"Token Count: {result['token_count']}")
    print(f"Tokens/Second: {result['tokens_per_second']:.2f}")

print("\nTest completed.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Original Model Evaluation Results
Average Inference Time: 27.22 seconds
Average Tokens/Second: 25.10

Full Outputs:

----------------------------------------
Sample 1:
Instruction: 서울의 유명한 관광 코스를 만들어줄래?
Generated Text:
물론이죠! 서울에는 다양한 문화와 역사, 자연과 현대적인 매력을 동시에 느낄 수 있는 많은 관광지가 있습니다. 여기 몇 가지 추천하는 서울의 유망 관광코스를 소개합니다:

### 코스 1: 역사와 문화 탐방
1. **경복궁**
   - 한국의 대표적인 궁궐로, 조선 왕조의 중심지였다.
   
2. **북촌 한옥마을**
   전통 한옥들이 잘 보존되어 있어 한국의 전통 생활상을 느껴볼 수 있는 곳.

3. **인사동**
   독특한 전통 공예품과 미술품을 구경할 수 있으며, 전통 음식도 맛볼 수 있다.

4. **불국사**
   고려 시대의 사찰로, 유네스코 세계문화유산으로 지정되었다.

5. **남대문 시장**
   다양한 한국 전통 상품과 음식을 즐길 수 있는 대형 시장.

### 코斯 2: 현대 서울 탐방 (홍대, 이태원 등)
1. **
   **홍대 거리**
   젊음의 거리로 유명하며, 다양한 카페, 클럽, 갤러리 등을 찾을 수 있다.
   
   **이태원**
   다문화 지역으로 다양한 국제 요리를 맛볼 수도 있고, 외국인 관광객들의 모임지.
   
6. **강남 스타필드**
   쇼핑과 레저를 즐길수 있는 복합 쇼핑몰.

7. **COEX 마켓**
   국내외 다양한 소비재와 전통 시장 같은 분위기를 경험할 수 있는 쇼핑센터.

8. **한강공원**
    서울 시내에서 가장 긴 공원으로 자전거 타기나 산책하기 좋은 장소.

### 추가 팁:
- **교통**: 지하철 이용을 권장한다. 서울의 교통 체계가 매우 발달하여 쉽게 이동할 수 있다. T-Money 카드를 